## 18.3 Homework

Try different minimization methods in scipy on larger systems ($N$ up to 20), and show 
- 1 the average number of attempts to find the ground state
- 2 the time costs


### Optional
try to improve the code to make it run faster, analyze the most time consuming part and give your solution

In [25]:
from scipy.spatial.distance import cdist
from scipy.optimize import minimize
import numpy as np
import pandas as pd
import requests
import time

def LJ(r):
    r6 = np.power(r,6)
    r12 = np.power(r,12)
    LJ = 4*(1/r12 - 1/r6)
    return LJ

def init_pos(N, L=5): 
    return L*np.random.random_sample((N*3,))

def total_energy(positions): 
    E = 0
    N_atom = int(len(positions)/3) 
    for i in range(N_atom-1):
        for j in range(i+1, N_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            dist = np.linalg.norm(pos1-pos2)
            E += LJ(dist)
    return E

def get_true_LJ(N):
    url = "http://doye.chem.ox.ac.uk/jon/structures/LJ/points/"+str(N)
    names = ['x', 'y', 'z']
    dataset = pd.read_csv(url, names=names, delim_whitespace=True)
    pos = dataset.values
    pos = np.reshape(pos, [N*3,1])
    return total_energy(pos)

In [ ]:
N = 20 #number of atoms
attempts = 5 #max number of LJ Potential calculations
methods = ['CG','Nelder-Mead','TNC']

for i in range(1,N+1):
    if i%5 == 0:
        trueLJ = round(get_true_LJ(i),1) #obtains the ground state potential from accepted documentation
        print('accepted LJ energy at N =',i,'is', trueLJ)
        print('calculated LJ energy at N =',i)
        n = 0 #this is the number of tries to find the best calculation on the LJ potential
        for j in methods:
            calculations = [] #list of all calculated LJ potentials
            times = [] #list of the times LJs were calculate
            
            time_start = time.time() #time start
            
            #this calculation is the most expensive
            res = minimize(total_energy, init_pos(i), method=j, tol=1e-4) 
            calcLJ = round(res.fun,1)
            
            times.append(time.time()) #calculated time is time[i]-time_start 
            calculations.append(calcLJ)
            
            count = 1 #this is where the count starts for number of attempts
            while calcLJ != trueLJ:
                #more expensive calculations
                res = minimize(total_energy, init_pos(i), method=j, tol=1e-4)
                calcLJ = round(res.fun,1) 
                
                calculations.append(calcLJ) 
                times.append(time.time())
                count += 1
                
                if count == attempts:
                    print('5 attempts made, best estimate on ground state is',min(calculations))
                    break
                    
            n += calculations.index(min(calculations))+1 #this is the number of tries to find the best calculation on the LJ potential
            
            print(min(calculations),'using',j,'took',calculations.index(min(calculations))+1,'tries and took',round(times[calculations.index(min(calculations))]-time_start,2),'seconds')
        print('average number of attempts to find the ground state:',round(n/len(methods),2))
        print('\n')

accepted LJ energy at N = 5 is -9.1
calculated LJ energy at N = 5
-9.1 using CG took 1 tries and took 0.75 seconds
5 attempts made, best estimate on ground state is -6.1
-6.1 using Nelder-Mead took 5 tries and took 2.73 seconds
5 attempts made, best estimate on ground state is -8.1
-8.1 using TNC took 1 tries and took 0.59 seconds
average number of attempts to find the ground state: 2.33


accepted LJ energy at N = 10 is -28.4
calculated LJ energy at N = 10
5 attempts made, best estimate on ground state is -27.6
-27.6 using CG took 3 tries and took 19.47 seconds
5 attempts made, best estimate on ground state is -17.2
-17.2 using Nelder-Mead took 3 tries and took 14.75 seconds
5 attempts made, best estimate on ground state is -14.0
-14.0 using TNC took 3 tries and took 21.64 seconds
average number of attempts to find the ground state: 3.0


accepted LJ energy at N = 15 is -52.3
calculated LJ energy at N = 15
5 attempts made, best estimate on ground state is -49.5
-49.5 using CG took 4 t

### i will attempt the optional part if i'm able to finish the rest of the assignments before the semester starts.  N= 20 WOULD NOT FINISH FOR ME